## Señales

Las señales en PyQt5 son de suma importancia, estas permiten que puedas tener multiples Threads corriendo y puedan comunicarse entre si sin chocar. Se puede entender a las señales como mensajes que le comunican a un Thread que otro le está pidiendo que haga algo. El uso de señales permite que cada Thread tenga manejo sobre lo que está ocurriendo en su ejecución.

### ¿Como implemento señales en PyQt5?

Muchos de los objetos de PyQt5 tienen sus propias señales. Por ejemplo, los `QPushButton` tiene una señal llamada `clicked` que se activa cuando se clickea el botón. 

¿Como las señales pueden comunicarle a otro Thread que han sido activadas?

Todas las señales tienen un método llamado `connect()`, que conecta la señal con la función que se entrega como párametro. De esta manera, para conectar la señal de clickeo de un QPushButton bastaría con escribir `my_button.clicked.connect(my_function)`.

Siempre es bueno que usen las señales de los objetos con los que están trabajado. Por ejemplo, si tienen un QLineEdit, la señal `returnPressed` se activa cuando la tecla Enter es oprimida, por lo que puede ser muy útil conectarla a una función que procese el texto que escribieron en el QLineEdit.

### Usar señales personalizadas

Puedes usar señales personalizadas cuando estes trabajando con cualquier QObject. Estas te permiten activarlas cuando lo necesites y enviar lo que quieras. Las señales son de la clase ´pyqtSignal´ que se encuentra en el módulo `PyQt5.Qt`.

Cuando instancias una señal debes indicarle el tipo de objeto que esta enviará cuando sea activada. Para activar la señal se utiliza el método `emit`, que recibe un objeto del tipo que se le indicó al ser instanciada, y llama a la función a la cual está conectada entregandole el objeto que recibe `emit`. Esto se puede ver a continuación.

In [1]:
from PyQt5.QtWidgets import QMainWindow, QApplication
from PyQt5.QtCore import pyqtSignal
    
class MyWindow(QMainWindow):
    
    # Se le indica a la señal el tipo de objeto que enviará, en este caso un string.
    my_signal = pyqtSignal(str)
    
    def __init__(self):
        super().__init__()
        # Conectamos la señal a la función my_function.
        self.my_signal.connect(self.my_function)
        
        # Activamos la señal 2 veces
        self.my_signal.emit("Primera señal")
        self.my_signal.emit("Segunda señal")
    
    def my_function(self, string):
        print("Me han llamado")
        print(string)

if __name__ == '__main__':
    app = QApplication([])
    ex = MyWindow()
    ex.setGeometry(100,100,500,500)
    ex.show()
    app.exec_()


Me han llamado
Primera señal
Me han llamado
Segunda señal


### Importancia de las señales

[Texto justificando importancia de señales]

Escribamos un programa en el que aparezcan circulos que se muevan cada cierto tiempo de 3 maneras distintas:

1-. Sin usar señales.

2-. Usando Señales.

3-. Usando Señales y eliminando `QThreads` con `DeleteLater`.

### Programa sin uso de señales


In [1]:
from PyQt5.QtWidgets import QLabel, QMainWindow, QApplication
from PyQt5.QtCore import QThread, QTimer
from PyQt5.QtGui import QPixmap
from time import sleep
from random import randint


class Food(QThread):

    def __init__(self, parent, x, y, max_width, max_height):
        """
        Un Food es un QThread que movera una imagen de comida
        en una ventana. El __init__ recibe los parametros:
            parent: ventana
            x e y: posicion inicial en la ventana
        """
        super().__init__()
        # Guardamos el path de la imagen que tendrá el Label
        self.food_image = "assets/{}.png".format(randint(1, 9))
        # Creamos el Label y definimos su tamaño
        self.label = QLabel(parent)
        self.label.setGeometry(x, y, 50, 50)
        self.label.setPixmap(QPixmap(self.food_image))
        self.label.setScaledContents(True)
        self.label.show()
        self.label.setVisible(True)
        # Seteamos la posición inicial y la guardamos para usarla como una property
        self.__position = (0, 0)
        self.position = (x, y)
        #Guardamos los limites de la ventana para que no pueda salirse de ella
        self.max_width = max_width
        self.max_height = max_height
        self.start()

    @property
    def position(self):
        return self.__position

    @position.setter
    def position(self, value):
        self.__position = value
        self.label.move(self.position[0], self.position[1])

    def run(self):
        while True:
            sleep(0.1)
            new_x = self.position[0] + 1
            if new_x > self.max_width:
                new_x = randint(0, self.max_width)
            new_y = self.position[1] + 1
            if new_y > self.max_height:
                new_y = randint(0, self.max_height)
            self.position = (new_x, new_y)


class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.titulo = QLabel(self)
        self.titulo.setText("Ejemplo")
        self.titulo.move(160, 10)
        self.titulo.show()
        self.setGeometry(100, 100, 1200, 800)
        self.show()

        # Contador de cuanta comida hemos creado
        self.food_created = 0

        # Creamos un Timer que se encargara de crear la comida
        self.food_creator_timer = QTimer(self)
        self.food_creator_timer.timeout.connect(self.food_creator)
        self.food_creator_timer.start(50)

        self.foods = []

    def food_creator(self):
        new_food = Food(parent=self, x=randint(0, self.width()),
                        y=randint(0, self.height()), max_width=self.width(),
                        max_height=self.height())
        self.foods.append(new_food)
        self.food_created += 1
        print("Has creado {} unidades de comida\n".format(self.food_created))

if __name__ == '__main__':
    app = QApplication([])
    ex = MyWindow()
    app.exec_()


Has creado 1 unidades de comida

Has creado 2 unidades de comida

Has creado 3 unidades de comida

Has creado 4 unidades de comida

Has creado 5 unidades de comida

Has creado 6 unidades de comida

Has creado 7 unidades de comida

Has creado 8 unidades de comida

Has creado 9 unidades de comida

Has creado 10 unidades de comida

Has creado 11 unidades de comida

Has creado 12 unidades de comida

Has creado 13 unidades de comida

Has creado 14 unidades de comida

Has creado 15 unidades de comida

Has creado 16 unidades de comida

Has creado 17 unidades de comida

Has creado 18 unidades de comida

Has creado 19 unidades de comida

Has creado 20 unidades de comida

Has creado 21 unidades de comida

Has creado 22 unidades de comida

Has creado 23 unidades de comida

Has creado 24 unidades de comida

Has creado 25 unidades de comida

Has creado 26 unidades de comida

Has creado 27 unidades de comida

Has creado 28 unidades de comida

Has creado 29 unidades de comida

Has creado 30 unidades 

Podemos observar que el programa se cae luego de crear cerca de 100 `QThreads`.

### Programa usando señales


In [ ]:
from PyQt5.QtWidgets import QLabel, QMainWindow, QApplication
from PyQt5.QtCore import QThread, pyqtSignal, QTimer
from PyQt5.QtGui import QPixmap
from time import sleep
from random import randint

class MoveMyImageEvent:
    """
    Las instancias de esta clase
    contienen la informacion necesaria
    para que la ventana actualice
    la posicion de la imagen
    """

    def __init__(self, label, x, y):
        self.label = label
        self.x = x
        self.y = y


class Food(QThread):
    trigger = pyqtSignal(MoveMyImageEvent)
    # pyqtSignal recibe *args que le indican
    # cuales son los tipos de argumentos que seran enviados
    # en este caso, solo se enviara un argumento:
    # objeto clase MoveMyImageEvent, podria ser tambien int o str en caso
    # de que busquemos enviar este tipo de objetos en la señal.

    def __init__(self, parent, x, y, max_width, max_height):
        """
        Un Food es un QThread que movera una imagen de comida
        en una ventana. El __init__ recibe los parametros:
            parent: ventana
            x e y: posicion inicial en la ventana
        """
        super().__init__()
        # Guardamos el path de la imagen que tendrá el Label
        self.food_image = "assets/{}.png".format(randint(1, 9))
        # Creamos el Label y definimos su tamaño
        self.label = QLabel(parent)
        self.label.setGeometry(x, y, 50, 50)
        self.label.setPixmap(QPixmap(self.food_image))
        self.label.setScaledContents(True)
        self.label.show()
        self.label.setVisible(True)
        # Seteamos la posición inicial y la guardamos para usarla como una property
        self.__position = (0, 0)
        self.position = (x, y)
        #Guardamos los limites de la ventana para que no pueda salirse de ella
        self.max_width = max_width
        self.max_height = max_height
        #self.trigger.connect(parent.actualizar_imagen)
        self.start()

    @property
    def position(self):
        return self.__position

    @position.setter
    def position(self, value):
        self.__position = value

        # El trigger emite su señal a la ventana cuando cambiamos la posición
        self.trigger.emit(MoveMyImageEvent(
            self.label, self.position[0], self.position[1]
        ))

    def run(self):
        while True:
            sleep(0.1)
            new_x = self.position[0] + 1
            if new_x > self.max_width:
                new_x = randint(0, self.max_width)
            new_y = self.position[1] + 1
            if new_y > self.max_height:
                new_y = randint(0, self.max_height)
            self.position = (new_x, new_y)


class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.titulo = QLabel(self)
        self.titulo.setText("Ejemplo")
        self.titulo.move(160, 10)
        self.titulo.show()
        self.setGeometry(100, 100, 1200, 800)
        self.show()

        # Contador de cuanta comida hemos creado
        self.food_created = 0

        # Creamos un Timer que se encargara de crear la comida
        self.food_creator_timer = QTimer(self)
        self.food_creator_timer.timeout.connect(self.food_creator)
        self.food_creator_timer.start(50)

        self.foods = []

    def food_creator(self):
        new_food = Food(parent=self, x=randint(0, self.width()),
                        y=randint(0, self.height()), max_width=self.width(),
                        max_height=self.height())
        new_food.trigger.connect(self.actualizar_imagen)
        self.foods.append(new_food)
        self.food_created += 1
        print("Has creado {} unidades de comida\n".format(self.food_created))

    @staticmethod
    def actualizar_imagen(myImageEvent):
        # Recibo el objeto con la información necesaria para mover a la comida
        label = myImageEvent.label
        label.move(myImageEvent.x, myImageEvent.y)

if __name__ == '__main__':
    app = QApplication([])
    ex = MyWindow()
    app.exec_()


Podemos observar que el programa se cae luego de crear cerca de 200 `QThreads`, lo cual es casi el doble que sin señales.

### Programa con señales y eliminando `QThreads` con `DeleteLater`.

In [1]:
from PyQt5.QtWidgets import QLabel, QMainWindow, QApplication
from PyQt5.QtCore import QThread, pyqtSignal, QTimer
from PyQt5.QtGui import QPixmap
from time import sleep
from random import randint


class MoveMyImageEvent:
    """
    Las instancias de esta clase
    contienen la informacion necesaria
    para que la ventana actualice
    la posicion de la imagen
    """
    def __init__(self, label, x, y):
        self.label = label
        self.x = x
        self.y = y


class Food(QThread):
    trigger = pyqtSignal(MoveMyImageEvent)

    # pyqtSignal recibe *args que le indican
    # cuales son los tipos de argumentos que seran enviados
    # en este caso, solo se enviara un argumento:
    #   objeto clase MoveMyImageEv

    def __init__(self, parent, x, y, max_width, max_height):
        """
        Un Food es un QThread que movera una imagen de comida
        en una ventana. El __init__ recibe los parametros:
            parent: ventana
            x e y: posicion inicial en la ventana
        """
        super().__init__()
        self.life = 100
        # Guardamos el path de la imagen que tendrá el Label
        self.food_image = "assets/{}.png".format(randint(1, 9))
        # Creamos el Label y definimos su tamaño
        self.label = QLabel(parent)
        self.label.setGeometry(x, y, 50, 50)
        self.label.setPixmap(QPixmap(self.food_image))
        self.label.setScaledContents(True)
        self.label.show()
        self.label.setVisible(True)
        # Seteamos la posición inicial y la guardamos para usarla como una property
        self.__position = (0, 0)
        self.position = (x, y)
        # Guardamos los limites de la ventana para que no pueda salirse de ella
        self.max_width = max_width
        self.max_height = max_height
        self.start()

    @property
    def position(self):
        return self.__position

    @position.setter
    def position(self, value):
        self.__position = value

        # El trigger emite su señal a la ventana cuando cambiamos la posición
        self.trigger.emit(MoveMyImageEvent(
            self.label, self.position[0], self.position[1]
        ))


    def run(self):

        while self.life > 0:
            self.life -= 1
            sleep(0.1)
            new_x = self.position[0] + 1
            if new_x > self.max_width:
                new_x = randint(0, self.max_width)
            new_y = self.position[1] + 1
            if new_y > self.max_height:
                new_y = randint(0, self.max_height)
            self.position = (new_x, new_y)
        self.quit()
        self.label.deleteLater()


class MyWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.titulo = QLabel(self)
        self.titulo.setText("Ejemplo")
        self.titulo.move(160, 10)
        self.titulo.show()
        self.setGeometry(100, 100, 1200, 800)
        self.show()

        # Contador de cuanta comida hemos creado
        self.food_created = 0

        # Creamos un Timer que se encargara de crear la comida
        self.food_creator_timer = QTimer(self)
        self.food_creator_timer.timeout.connect(self.food_creator)
        self.food_creator_timer.start(50)

        self.foods = []

    def food_creator(self):
        new_food = Food(parent=self, x=randint(0, self.width()),
                        y=randint(0, self.height()), max_width=self.width(),
                        max_height=self.height())
        new_food.trigger.connect(self.actualizar_imagen)
        self.foods.append(new_food)
        self.food_created += 1
        print("Has creado {} unidades de comida\n".format(self.food_created))

    @staticmethod
    def actualizar_imagen(myImageEvent):
        # Recibo el objeto con la información necesaria para mover a la comida
        label = myImageEvent.label
        label.move(myImageEvent.x, myImageEvent.y)


if __name__ == '__main__':
    app = QApplication([])
    ex = MyWindow()
    app.exec_()


Has creado 1 unidades de comida

Has creado 2 unidades de comida

Has creado 3 unidades de comida

Has creado 4 unidades de comida

Has creado 5 unidades de comida

Has creado 6 unidades de comida

Has creado 7 unidades de comida

Has creado 8 unidades de comida

Has creado 9 unidades de comida

Has creado 10 unidades de comida

Has creado 11 unidades de comida

Has creado 12 unidades de comida

Has creado 13 unidades de comida

Has creado 14 unidades de comida

Has creado 15 unidades de comida

Has creado 16 unidades de comida

Has creado 17 unidades de comida

Has creado 18 unidades de comida

Has creado 19 unidades de comida

Has creado 20 unidades de comida



KeyboardInterrupt: 

Has creado 21 unidades de comida

Has creado 22 unidades de comida

Has creado 23 unidades de comida

Has creado 24 unidades de comida

Has creado 25 unidades de comida

Has creado 26 unidades de comida

Has creado 27 unidades de comida

Has creado 28 unidades de comida

Has creado 29 unidades de comida

Has creado 30 unidades de comida

Has creado 31 unidades de comida

Has creado 32 unidades de comida

Has creado 33 unidades de comida

Has creado 34 unidades de comida

Has creado 35 unidades de comida

Has creado 36 unidades de comida

Has creado 37 unidades de comida

Has creado 38 unidades de comida

Has creado 39 unidades de comida

Has creado 40 unidades de comida

Has creado 41 unidades de comida

Has creado 42 unidades de comida

Has creado 43 unidades de comida

Has creado 44 unidades de comida

Has creado 45 unidades de comida

Has creado 46 unidades de comida

Has creado 47 unidades de comida

Has creado 48 unidades de comida

Has creado 49 unidades de comida

Has creado 50 

Podemos observar que el programa puede crear más de 500 `QThreads` sin caerse, por lo que se hace evidente que siempre es bueno usar señales, junto con ir eliminando del programa aquellos QThreads y QObjects que ya no son necesa